<a href="https://colab.research.google.com/github/satyasri77/chat_bot/blob/main/Chatbot_for_cust_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain openai faiss-cpu python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.1 MB/s eta 0:00:00


In [ ]:
!unzip -q Ollama.app.zip -d Ollama_app

In [ ]:
!pip install -r Ollama_app/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Ollama_app/requirements.txt'


In [ ]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


In [2]:
load_dotenv()
OPENAI_API_KEY="your-key"

In [ ]:
# Step 1: Load and process your documents
loader = TextLoader("/content/customer_service_info.txt")  # Replace with your file
documents = loader.load()

In [ ]:
documents

[Document(metadata={'source': '/content/customer_service_info.txt'}, page_content='## General Information\nOur company provides a variety of products and services designed to meet the needs of both individual and business customers. This document outlines key support policies and answers frequently asked questions.\n\n---\n\n## Contact Information\nYou can reach our customer support team in the following ways:\n- 📧 Email: support@yourcompany.com\n- 📞 Phone: 1-800-555-1234\n- 💬 Live Chat: Available on our website during business hours.\n\nBusiness Hours:\nMonday to Friday, 9 AM to 6 PM EST\n\n---\n\n## Return & Refund Policy\nCustomers may return items within 30 days of receipt. The following conditions apply:\n- Items must be unused and in their original packaging.\n- Proof of purchase is required.\n- Refunds are processed within 5–7 business days after receiving the returned item.\n- Certain items (e.g., gift cards, clearance products) may be non-refundable.\n\nFor return instructions

In [ ]:
# Step 2: Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)


In [ ]:
# Step 3: Create embeddings and store in vector DB (FAISS)
# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# db = FAISS.from_documents(docs, embeddings)


from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



/tmp/ipython-input-11-4008684360.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
db

In [ ]:
from langchain.prompts import PromptTemplate
custom_prompt = PromptTemplate.from_template("""
You are a helpful customer support assistant.

You must ONLY answer using the information provided in the context below.
If the answer is not in the context, reply: "I don't know based on the information provided."

Context:
{context}

Question: {question}
Answer:
""")

In [ ]:
# Step 4: Set up Retrieval-based QA
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

/tmp/ipython-input-14-2481940358.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0),


In [ ]:
# Step 5: Ask a question
def ask(question):
    result = qa_chain({"query": question})
    return result["result"]


In [ ]:
# Example
if __name__ == "__main__":
    while True:
        q = input("User: ")
        if q.lower() in ["exit", "quit"]:
            break
        print("Bot:", ask(q))

User: what's the return policy?


/tmp/ipython-input-15-1835266445.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}